# template_title validation using GLODAP

In [ ]:
chunk_start

In [ ]:
variable = "template_variable" 
Variable = variable.title()

In [ ]:
if variable == "ph":
    Variable = "pH"
else:   
    Variable = variable.upper()

In [ ]:
md(f"{Variable} was validated using the 2016 GLODAP climatology. The GLODAP climatology is a collection of the best available data for the global ocean carbon cycle. The climatology is based on the GLODAPv2 data product, which is described in detail in Olsen et al. (2016). The GLODAPv2 data product is available at https://www.glodap.info/index.php/mapped-data-product/. The GLODAP climatology is available at https://www.glodap.info/glodap-climatology-2016.html. The GLODAP climatology is a collection of the best available data for the global ocean carbon cycle. The climatology is based on the GLODAPv2 data product, which is described in detail in Olsen et al. (2016).") 
md("Model data was regridded to the spatial resolution of the GLODAP product (1 degree by 1 degree) and then compared to the GLODAP climatology. The GLODAP climatology was interpolated to the model grid using bilinear interpolation. The GLODAP climatology is based on data for the years 1972-2013. The model data was averaged over the same time period. We only considered to sea surface")

In [ ]:
ds_model = nc.open_data(f"../../matched/gridded/glodap/glodap_{variable.lower()}.nc")
mask_all(ds_model)
ds_model.subset(variable = "model")
ds_model.as_missing(0)
ds_model.run()
ds_annual = ds_model.copy()
ds_annual.tmean()
ds_annual.set_longnames({"model": Variable})

In [ ]:
ds_obs = nc.open_data(f"../../matched/gridded/glodap/glodap_{variable.lower()}.nc")
ds_obs.subset(variable = "observation")
ds_obs.tmean("month")
ds_obs.as_missing(0)
ds_obs.run()
ds_obs.tmean()
ds_obs.set_longnames({"observation": Variable})

In [ ]:
ds_obs = nc.open_data(f"../../matched/gridded/glodap/glodap_{variable.lower()}.nc")
ds_obs.subset(variable = "observation")
ds_obs.run()
model_units = ds_model.contents.unit.values[0]
bad_unit = True
obs_units = ds_obs.contents.unit.values[0]


ds_obs.regrid(ds_model)
ds_obs.run()
ds_obs.top()
ds_obs.run()
obs_mask = ds_obs.copy()
obs_mask > -1e20
mod_mask = ds_model.copy()
mod_mask > -1e20
mod_mask * obs_mask
mod_mask.run()
ds_model * mod_mask
ds_obs * mod_mask

In [ ]:
chunk_clim

In [ ]:
chunk_bias

In [ ]:
ds_annual = ds_model.copy()
ds_annual.rename({ds_annual.variables[0]: "model"})
ds_annual.append(ds_obs)
ds_annual.tmean()
ds_annual.merge("variables")
ds_annual.rename({ds_obs.variables[0]: "observation"})
out_dir = "../../results/annual_mean/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
out_file = out_dir + f"annualmean_{variable}.nc"
ds_annual.to_nc(out_file, zip = True, overwrite = True)

In [ ]:
chunk_end